# Intro
Klik bij iedere cell op 'Run'. Controleer de uitvoer. Notebooks geven een stap-voor-stap uitvoering van code, waardoor gekeken kan worden of het goed verloopt.

In [6]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET

def to_xml(df, file_name=None, root_name='root', row_name='row', index_name='index', do_index=False, use_atts=False):
    root = ET.Element(root_name)
    tree = ET.ElementTree(root)
    for idx, row in df.iterrows():
        rowelm = ET.SubElement(root, row_name)
        if do_index:
            if use_atts:
                rowelm.set(index_name, str(idx))
            else:
                colelm = ET.SubElement(rowelm, index_name)
                colelm.text = str(idx)
        for hdr, val in row.items():
            hdr = hdr.replace(' ', '_')
            if pd.notnull(val):
                val = str(val)
            else:
                val = ''
            if use_atts:
                rowelm.set(hdr, str(val))
            else:
                colelm = ET.SubElement(rowelm, hdr)
                colelm.text = str(val)
    if file_name is not None:
        tree.write(file_name, xml_declaration=False, encoding='utf8')
    return ET.tostring(root)

# Lees prerelease
De prerelease HTML tabel wordt van de LOINC site gelezen. Het kan even duren.

Controleer de output! Verwacht wordt een tabel zoals op de LOINC site te zien, als de LOINC HTML wijzigt kan dit niet goed gaan.

(Noot: de sumission queue heeft meer details, mogelijk kan dit ook een bron zijn.)

In [7]:
uri = 'https://loinc.org/prerelease/'
# uri = 'https://loinc.org/submissions/queue/'
prerelease = pd.read_html(uri)[0]
prerelease

,Created On,LOINC,Long Common Name,Special Use,Component,Prop,Time,System,Scale,Method,Shortname
0,2023-09-22,103557-5,SARS-CoV-2 (COVID-19) Mpro gene mutation detec...,Special Use codes are developed in response to...,SARS coronavirus 2 Mpro gene mutation detected,Prid,Pt,XXX,Nom,Molgen,SARS-CoV-2 Mpro gene mut Spec
1,2023-08-23,103556-7,PLA2G7 and PLAC8 gene expression level [# Rati...,NaN,PLA2G7 & PLAC8 gene expression level,NRto,Pt,Bld,Qn,Molgen,NaN
2,2023-08-15,103555-9,Escherichia coli O157:H7 DNA [Presence] in Wou...,NaN,Escherichia coli O157:H7 DNA,PrThr,Pt,Wound,Ord,Probe.amp.tar,NaN


# Formatteren van de headers

In [8]:
prerelease.drop(['Created On', 'Special Use'], axis=1, inplace=True)
prerelease.columns = ['LOINC', 'longName', 'component', 'property', 'timing', 'system', 'scale', 'method', 'shortName']
prerelease.head()

,LOINC,longName,component,property,timing,system,scale,method,shortName
0,103557-5,SARS-CoV-2 (COVID-19) Mpro gene mutation detec...,SARS coronavirus 2 Mpro gene mutation detected,Prid,Pt,XXX,Nom,Molgen,SARS-CoV-2 Mpro gene mut Spec
1,103556-7,PLA2G7 and PLAC8 gene expression level [# Rati...,PLA2G7 & PLAC8 gene expression level,NRto,Pt,Bld,Qn,Molgen,NaN
2,103555-9,Escherichia coli O157:H7 DNA [Presence] in Wou...,Escherichia coli O157:H7 DNA,PrThr,Pt,Wound,Ord,Probe.amp.tar,NaN


# Maak XML
Het dataframe wordt geëxporteerd naar XML.
Vul target, usernaam, password in. Let op: eerst de cell runnen, dan waarden invullen. Cell niet opnieuw runnen, dat zet de default waarden terug.

In [9]:
xml = to_xml(prerelease, 'prerelease.xml', root_name='prerelease', row_name='concept')

In [19]:
import ipywidgets as widgets
targeturi = widgets.Dropdown(
    options=[('Productie', 'https://labterminologie.nl/lab/prerelease'),
             ('Acceptatie', 'https://terminologie.test-nictiz.nl/lab/prerelease'),
             ('Lokaal', 'http://localhost:8877/exist/apps/lab-data/data/prerelease.xml')],
    description='Target:',
)
usr = widgets.Text(description='Username')
pwd = widgets.Password(description='Password')
widgets.VBox([
  targeturi, usr, pwd
])

In [20]:
targeturi.value

'http://localhost:8877/exist/apps/lab-data/data/prerelease.xml'


# Upload naar de server
Verwachte uitvoer: (HTTP status) 201, dat is 'Created'

In [21]:
resp = requests.put(targeturi.value, data=xml, auth=requests.auth.HTTPBasicAuth(usr.value, pwd.value))
pwd.value = ''
resp.status_code

201

# Controleer de uitvoer
Onderstaande links voeren naar de prerelease tabel zoals zojuist geüpload.

[prerelease productie](http://labterminologie.nl/lab/prerelease)

[prerelease acceptatie](https://terminologie.test-nictiz.nl/lab/prerelease)

[prerelease lokaal](http://localhost:8877/exist/apps/lab-data/data/prerelease.xml)